In [4]:
import pandas as pd
import numpy as np

from my_functions_idealisedprofiles import *

### Break down analysis for Kippax and Garforth

In [5]:
region = 'Garforth' # 'Kippax' #'' # 'Garforth'

### Get version of landcover array with just 'urban' and 'rural' categories

In [6]:
# Read in the data
landcover, out_meta = prepare_rainfall_scenario_raster("../../../../FloodModelling/LandCoverData/{}/LandCover_clipped.tif".format(region), True)
# Convert the 1 and 6 values to 10 (for urban) and the rest to 11 (for non-urban).  
landcover_mod =  np.where(landcover==1, 10, landcover)
landcover_mod =  np.where(landcover_mod==6, 10, landcover_mod)
# Convert the rest of the classes to 11
for i in [1,2,3,4,5,7,8,9]:
    landcover_mod =  np.where(landcover_mod==i, 11, landcover_mod)

### Define the names of the method (shorter and longer versions)

In [7]:
methods =[ '6h_sp_c_0.5','6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4',
          '6h_sp_bl_0.6','6h_sp_bl_0.7','6h_sp_bl_0.8','6h_sp_bl_0.9']

### Find maximum intensity for each method and minute in which it occurs (to use in sorting results analysis)

In [8]:
maxs = []
min_of_maxs = []

# Add FEH data
# feh_precip=pd.read_csv("../../CreateSyntheticRainfallEvents/ReFH2_singlepeak/6hr_100yrRP/PostLossRemoval/6h_feh_singlepeak_urban.csv")
# maxs.append(feh_precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
# min_of_maxs.append(feh_precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

for method in methods:
    precip=pd.read_csv("../../CreateSyntheticRainfallEvents/IdealisedProfiles/6hr_100yrRP/PostLossRemoval/{}_urban.csv".format(method))
    # Trim and add minutes column
    precip = precip[0:360].copy()
    precip['minute']=range(1,361)
    # Add max and minutes of max
    maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
    min_of_maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

### Create versions of lists of methods, in order based on max intensity and the the timing of the max intensity 

In [9]:
short_ids_by_loading=  pd.DataFrame({"min": min_of_maxs, 'method_name': methods}).sort_values('min')["method_name"].tolist()
short_ids_by_intensity = pd.DataFrame({"min": maxs, 'method_name': methods}).sort_values('min', ascending = False)["method_name"].tolist()

### Create dataframe of colours for each cluster (based on their loading)

In [10]:
colours_df = create_colours_df(short_ids_by_loading, methods)

### Create list of filepaths, formatted to be used for either depth or velocity

In [11]:
fps = []
for method_num, short_id in enumerate(methods):
    fp = model_directory + "{}/{}/{} (Max).Resampled.Terrain.tif".format(short_id, region, '{}')
    fps.append(fp)

### Define breaks for categorising velocity and depth

In [12]:
# Define breaks to split the depths/velocities on
breaks_depths = np.array([0, 0.3, 0.6, 1.2, 100])  
labels_depth = ['<=0.3m', '0.3-0.6m', '0.6-1.2m', '>1.2m']
breaks_velocity = np.array([0,0.25,0.5,2,100])
labels_velocity = ["<=0.25m/s", "0.25-0.5m/s", "0.5-2m/s", ">2m/s"]

# <u> Flood extent </u>
To examine whether the rainfall's temporal distribution influences the total extent of flooding, the number of flooded cells and the total flooded area in km2 (incl. only cells with depth >0.1m) is compared between the profile with a single peak, and the three methods for producing multi-peaked rainfall events. b

### Create dataframes containing the (total/urban) flooded area in each depth/velocity bin

In [13]:
velocity_counts, velocity_props = create_binned_counts_and_props(fps, 'Velocity', breaks_velocity, labels_velocity, remove_little_values)
depth_counts, depth_props = create_binned_counts_and_props(fps, 'Depth', breaks_depths, labels_depth, remove_little_values)

In [14]:
velocity_counts_urban, velocity_props_urban = create_binned_counts_and_props_urban(fps, 'Velocity', breaks_velocity, labels_velocity, remove_little_values, landcover_mod)
depth_counts_urban, depth_props_urban = create_binned_counts_and_props_urban(fps, 'Depth', breaks_depths, labels_depth, remove_little_values, landcover_mod)

### Create dataframes containing the (total/urban) flooded area

In [15]:
totals_df = create_totals_df(velocity_counts)
totals_df_urban = create_totals_df(velocity_counts_urban)      

In [16]:
velocity_counts_urban

,index,6h_sp_c_0.5,6h_sp_fl_0.1,6h_sp_fl_0.2,6h_sp_fl_0.3,6h_sp_fl_0.4,6h_sp_bl_0.6,6h_sp_bl_0.7,6h_sp_bl_0.8,6h_sp_bl_0.9
0,<=0.25m/s,202431,193477,195696,198342,200513,204540,206255,208265,210021
1,0.25-0.5m/s,16393,9898,11328,12919,14567,18113,19699,20918,21648
2,0.5-2m/s,778,379,451,542,648,944,1111,1280,1361
3,>2m/s,0,0,0,0,0,0,0,0,0


### Create dataframes containing the % diff in the flooded area between single peak and each other method  

In [17]:
percent_diffs_df = find_percentage_diff (totals_df, fps) 
percent_diffs_df_urban = find_percentage_diff (totals_df_urban, fps)   

## Find number of cells in which each method leads to the worst flooding (depth/velocity)

In [18]:
# Find the number of flooded cells with the worst flooding for each method
worst_case_method_depth = find_worst_case_method(fps, methods, 'Depth')
worst_case_method_velocity = find_worst_case_method(fps, methods,  'Velocity') 

In [19]:
# Remove multiple matches and nan
worst_case_method_depth = worst_case_method_depth[~worst_case_method_depth['values'].isin(['multiple matches','nan'])]
worst_case_method_velocity = worst_case_method_velocity[~worst_case_method_velocity['values'].isin(['multiple matches','nan'])]

# # Reorder (and also add in the methods that are missing)
worst_case_method_depth = pd.merge(worst_case_method_depth,  pd.DataFrame({'values': methods}), how="outer")
worst_case_method_depth = worst_case_method_depth.reindex(worst_case_method_depth['values'].map(dict(zip(methods, range(len(methods))))).sort_values().index)
worst_case_method_depth.reset_index(inplace=True,drop=True)

worst_case_method_velocity = pd.merge(worst_case_method_velocity,  pd.DataFrame({'values': methods}), how="outer")
worst_case_method_velocity = worst_case_method_velocity.reindex(worst_case_method_velocity['values'].map(dict(zip(methods, range(len(methods))))).sort_values().index)
worst_case_method_velocity.reset_index(inplace=True,drop=True)

## Find number of cells with each hazard rating

In [20]:
hazard_counts, hazard_props = create_binned_counts_and_props_hazard(fps)

## Find number of cells which have moved between hazard categories

In [21]:
hazard_cat_changes = create_binned_counts_and_props_hazard_cat_change(fps)

### Create a dataframe containing all the info on each of the scenarios

In [22]:
cluster_results = pd.DataFrame({'Cluster_num': methods, "MaxRainfallIntensity": maxs,  
    "MaxRainfallIntensityMinute": min_of_maxs,
   'TotalFloodedArea':totals_df['FloodedArea'],'%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs'],
    '%Diff_FloodedArea_fromSP_formatted':percent_diffs_df['percent_diff_formatted'],
    'Abs%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs_abs'],'UrbanFloodedArea':totals_df_urban['FloodedArea'],
  '%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs'] ,
  '%Diff_UrbanFloodedArea_fromSP_formatted':percent_diffs_df_urban['percent_diff_formatted'],
    'Abs%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs_abs'], 
    'WorstCaseDepth_ncells': worst_case_method_depth['counts'].tolist(),
    'WorstCaseVelocity_ncells': worst_case_method_velocity['counts'].tolist(), 
                                'colour':colours_df['colour']}) 

In [23]:
cluster_results

,Cluster_num,MaxRainfallIntensity,MaxRainfallIntensityMinute,TotalFloodedArea,%Diff_FloodedArea_fromSP,%Diff_FloodedArea_fromSP_formatted,Abs%Diff_FloodedArea_fromSP,UrbanFloodedArea,%Diff_UrbanFloodedArea_fromSP,%Diff_UrbanFloodedArea_fromSP_formatted,Abs%Diff_UrbanFloodedArea_fromSP,WorstCaseDepth_ncells,WorstCaseVelocity_ncells,colour
0,6h_sp_c_0.5,0.301370,180,0.323883,0.00,,0.00,0.219602,0.00,,0.00,1420,11448,grey
1,6h_sp_fl_0.1,0.232995,36,0.301875,-6.80,-6.8%,6.80,0.203754,-7.22,-7.22%,7.22,12667,36653,darkblue
2,6h_sp_fl_0.2,0.250089,72,0.306982,-5.22,-5.22%,5.22,0.207475,-5.52,-5.52%,5.52,23909,21051,darkblue
3,6h_sp_fl_0.3,0.267183,108,0.312887,-3.40,-3.4%,3.40,0.211803,-3.55,-3.55%,3.55,25425,16921,paleturquoise
4,6h_sp_fl_0.4,0.284277,144,0.318379,-1.70,-1.7%,1.70,0.215728,-1.76,-1.76%,1.76,608,13705,paleturquoise
5,6h_sp_bl_0.6,0.318464,216,0.330594,2.07,+2.07%,2.07,0.223597,1.82,+1.82%,1.82,5276,9523,indianred
6,6h_sp_bl_0.7,0.335558,252,0.336859,4.01,+4.01%,4.01,0.227065,3.40,+3.4%,3.40,3964,9913,indianred
7,6h_sp_bl_0.8,0.352652,288,0.341844,5.55,+5.55%,5.55,0.230463,4.95,+4.95%,4.95,3663,9582,darkred
8,6h_sp_bl_0.9,0.369745,324,0.345601,6.71,+6.71%,6.71,0.233030,6.11,+6.11%,6.11,222881,171921,darkred


### Add the depth/velocity category breakdowns and hazard categories to this

In [24]:
dfs = [velocity_props, depth_props, velocity_props_urban, depth_props_urban, velocity_counts, depth_counts,
          velocity_counts_urban, depth_counts_urban,hazard_counts, hazard_props]
suffixes = ['_propcells', '_propcells','_propcells_urban','_propcells_urban','_countcells','_countcells','_countcells_urban','_countcells_urban',
'_numcells', '_propcells']

for num, df in enumerate(dfs):
    # Reformat the dataframe
    df = df.set_index('index').T
    # Add the correct suffix to the column names
    df = df.add_suffix(suffixes[num]) 
    # Add Cluster_num column for joining
    df['Cluster_num'] = df.index#
    # Join to cluster results dataframe
    cluster_results = pd.merge(cluster_results,  df, how="outer", on = 'Cluster_num')
    
cluster_results = pd.merge(cluster_results, hazard_cat_changes,  how="outer", on = 'Cluster_num')    

### Save to file

In [25]:
cluster_results.to_csv("Data/{}allclusters_summary.csv".format(region), index=False)

### Delete tiff files (as these aren't used again and take up a lot of space)

In [26]:
# for method in short_ids:
#     print(method)
#     if method != '6h_feh_sp':
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/hazard_cat_difference.tif".format(method)) 
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_difffromsinglepeak_classified.tif".format(method)) 
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_difffromsinglepeak_posneg.tif".format(method)) 
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_difffromsinglepeak_classified.tif".format(method)) 
#         os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_difffromsinglepeak_posneg.tif".format(method)) 
        
#     os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_classified.tif".format(method)) 
#     os.remove("../../../../FloodModelling/MeganModel_New/{}/hazard_classified.tif".format(method)) 
#     os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_classified.tif".format(method)) 